In [ ]:
pip install bitsandbytes transformers[accelerate] gradio

In [ ]:
!pip install fastapi uvicorn pyngrok huggingface_hub torch

In [ ]:
!ngrok config add-authtoken <add-your-token>

In [ ]:
# Import necessary libraries

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from fastapi import FastAPI, Request
from pydantic import BaseModel
from huggingface_hub import login



# Authenticate with Hugging Face using your access token
login("<add-your-token>")  # Replace with your actual token

# Model configuration
model_name = "meta-llama/Llama-3.1-8B-Instruct"

# 4-bit quantization configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    torch_dtype=torch.float16,
    device_map="auto",
)


# Chat function
def chat_with_llama(prompt):
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1000).to("cuda")
    outputs = model.generate(
        inputs.input_ids,
        max_length=1500,
        temperature=0.7,
        top_k=50,
        do_sample=True,
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response


In [ ]:
pip install firebase-admin

In [ ]:
import firebase_admin
from firebase_admin import credentials, firestore

# Initialize the Firebase Admin SDK
cred = credentials.Certificate('/content/serviceAccountKey.json')
firebase_admin.initialize_app(cred)

In [ ]:
pip install fastapi[all]

In [ ]:
from fastapi import FastAPI, Request
from pydantic import BaseModel
from fastapi.middleware.cors import CORSMiddleware
import uvicorn
from pyngrok import ngrok  # For exposing the API to the internet
import nest_asyncio
import numpy as np
from typing import List, Optional

# Define weights for accuracy and improvement
ALPHA = 0.7
BETA = 0.3

db = firestore.client()

nest_asyncio.apply()
# Define the FastAPI app
app = FastAPI()

global conversation_history
conversation_history = []

class UserRequest(BaseModel):
    user_id: str


# Add CORS middleware to allow requests from React (or other web apps)
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # This allows requests from any origin. Change it for more security.
    allow_credentials=True,
    allow_methods=["*"],  # Allows all methods (GET, POST, etc.)
    allow_headers=["*"],  # Allows all headers
)

def select_questions(fitness_score):
    # Thresholds can be adjusted as needed
    if fitness_score > 0.8:
        # Select mostly difficult questions
        difficulty_distribution = {'easy': 0, 'medium': 1, 'difficult': 2}
    elif fitness_score > 0.5:
        # Balanced selection
        difficulty_distribution = {'easy': 1, 'medium': 1, 'difficult': 1}
    else:
        # Easier questions
        difficulty_distribution = {'easy': 2, 'medium': 1, 'difficult': 0}


    return difficulty_distribution


def calculate_improvement(current_accuracy, past_accuracies):
    # Calculate the average of past accuracies
    past_average = np.mean(past_accuracies)
    print(past_average)
    return current_accuracy - past_average

# Request model for FastAPI
class ChatRequest(BaseModel):
    topic: str
    #custom_prompt: str = None
    custom_prompt: Optional[str] = None
    options: Optional[List[str]] = None  # Add this for quiz options
    correct_answer: Optional[str] = None  # Add this for the correct answer
    selected_answer: Optional[str] = None  # Add this for the selected answer

# Endpoint for quiz generation
@app.post("/generate_quiz")
async def generate_quiz(request: ChatRequest):
    prompt = f"""
    You are tutoring a 12-year-old student.
    Give 5 simple quiz questions each with 4 options for the following topic: {request.topic}
    AI:
    """
    response = chat_with_llama(prompt)
    return {"quiz": response.split("AI:")[1].strip()}

# Endpoint for general chat
@app.post("/ask_anything")
async def ask_anything(request: ChatRequest):


    prompt = f"""
        You are a helpful assistant tutoring a 12-year-old-student. Given the conversation that has been done already, respond to the following query:
        Conversation: {conversation_history}
        Query: {request.topic}
        AI:
    """
    print(prompt)
    response = chat_with_llama(prompt)
    print(response)
    # Add the new user query to the conversation history
    conversation_history.append(f"Student: {request.topic}")

    # Construct the full context including the conversation history
    #context = " ".join(conversation_history[-5:])  # Limit to last 5 exchanges for brevity
    # Add the model's response to the conversation history
    conversation_history.append(f"AI: {response}")

    return {"response": response.split("AI:")[-1].strip()}

# Endpoint for general chat
@app.post("/simplify")
async def simplify(request: ChatRequest):


    prompt = f"""
        You are a helpful assistant tutoring a 12-year-old-student. Given the content from a textbook about a topic simplify it:
        Textbook Content: {request.topic}
        AI:
    """
    print(prompt)
    response = chat_with_llama(prompt)
    print(response)

    return {"response": response.split("AI:")[-1].strip()}


@app.post("/quiz_explanation")
async def quiz_explanation(request: ChatRequest):
    prompt = f"""
    You are a helpful assistant tutoring a 12-year-old student. Given the quiz questions with options, correct answer, and option selected by the user, provide a detailed feedback on why the selected answer is wrong and explain the correct answer:
    Quiz question: {request.topic}
    Options: {', '.join(request.options)}
    Correct answer: {request.correct_answer}
    Selected answer: {request.selected_answer}
    AI:
    """
    print(prompt)
    response = chat_with_llama(prompt)
    print(response)

    return {"response": response.split("AI:")[-1].strip()}
    #return {"response": prompt}


#user_request: UserRequest
@app.post("/fitness_calculation")
async def fitness_calculation(user_request: UserRequest):
    user_id = user_request.user_id
    # Reference the specific user document
    doc_ref = db.collection('users').document(user_id)

    # Fetch the document
    doc = doc_ref.get()
    if doc.exists:
        data = doc.to_dict()
        quiz_scores = []

        # Iterate through the progress dictionary
        for key, value in data.get('progress', {}).items():
            if 'quizScore' in value:
                quiz_scores.append(value['quizScore']/100)

        improvement = calculate_improvement(quiz_scores[-1], quiz_scores)
        fitness_score = ALPHA * quiz_scores[-1] + BETA * improvement
        difficulty_distribution = select_questions(fitness_score)

        # Print all quiz scores
        for index, score in enumerate(quiz_scores, start=1):
            print(f"Quiz Score {index}: {score}")
    else:
        HTTPException(status_code=404, detail="User document not found")

    return {"difficulty_distribution": difficulty_distribution}


# Expose the API to the internet using ngrok
def expose_ngrok():
    url = ngrok.connect(8000)
    print(f"Public URL: {url}")

if __name__ == "__main__":
    # Start ngrok
    expose_ngrok()

    # Run the FastAPI app
    uvicorn.run(app, host="0.0.0.0", port=8000)